# Sistema de Recomendação Híbrido — **v4 (Otimizado)**

**Melhorias**: leitura em *chunks* para `interacoes.csv`, amostragem automática para bases enormes, normalização do `score` (0–1) no ensemble, tempo de execução por fase, TF‑IDF e NMF com parâmetros dinâmicos.

**Como usar**:
1) Rode a Fase 1 (imports). 2) Rode as Fases 2.1 → 2.3 (upload de cada CSV). 3) Rode a Fase 2.4 (checagem). 4) Rode a Fase 3 (validação/otimização). 5) Fases 4–7 para treinar, avaliar e exportar.


## Fase 1 — Configuração do Ambiente

In [13]:
# ===== Fase 1: Configurações e imports =====
from __future__ import annotations  # compat com type hints
import io  # bytes do upload
import os  # arquivos locais
import math  # utilidades matemáticas
import time  # medição de tempo
import logging  # logs
from typing import Dict, List, Tuple  # tipos

import numpy as np  # numérico
import pandas as pd  # dataframes

from sklearn.feature_extraction.text import TfidfVectorizer  # CBF
from sklearn.decomposition import NMF  # CF
from sklearn.preprocessing import MinMaxScaler  # normalização
from sklearn.metrics import mean_squared_error  # RMSE
from scipy.sparse import csr_matrix  # matriz esparsa

# Detecta Colab
try:
    from google.colab import files  # ferramentas Colab
    IN_COLAB = True  # flag
except Exception:
    IN_COLAB = False  # local

# Stopwords PT (opcional)
STOPWORDS_PT: List[str] | None = None  # container
try:
    import nltk  # NLP
    from nltk.corpus import stopwords  # stopwords
    try:
        _ = stopwords.words("portuguese")  # cache
    except LookupError:
        nltk.download("stopwords", quiet=True)  # baixa
    STOPWORDS_PT = stopwords.words("portuguese")  # define
except Exception:
    STOPWORDS_PT = None  # ignora

# Logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")  # setup
logger = logging.getLogger("recom_v4")  # logger

# Parâmetros
RANDOM_STATE = 42  # seed
np.random.seed(RANDOM_STATE)  # fixa seed

PESOS = {"cbf": 0.55, "cf": 0.35, "kb": 0.10}  # ensemble
MAX_TFIDF = 120_000  # features
BASE_CHUNKSIZE = 200_000  # chunk leitura
N_COMPONENTS_NMF = 32  # fatores


## Fase 2 — Upload dos Arquivos CSV (3 execuções independentes)

In [2]:
# ===== Fase 2.1: Upload do arquivo dados_alunos.csv =====
if not IN_COLAB:
    raise EnvironmentError("Esta célula foi projetada para o Google Colab.")  # garante Colab
print("📂 Selecione o arquivo: dados_alunos.csv")  # instrução
up = files.upload()  # janela
if "dados_alunos.csv" in up:  # valida
    alunos_df = pd.read_csv(io.BytesIO(up["dados_alunos.csv"]), encoding="utf-8", sep=",")  # lê
    alunos_df.to_csv("dados_alunos.csv", index=False, encoding="utf-8")  # salva
    print("✔ 'dados_alunos.csv' carregado e salvo.")  # feedback
    display(alunos_df.head(5))  # amostra
else:
    print("⚠️ Envie 'dados_alunos.csv' e reexecute esta célula.")  # aviso


📂 Selecione o arquivo: dados_alunos.csv


Saving dados_alunos.csv to dados_alunos.csv
✔ 'dados_alunos.csv' carregado e salvo.


,id_aluno,nome,curso,periodo,disciplinas_cursadas,areas_interesse
0,1,João Silva,Engenharia de Software,5,"Algoritmos, Banco de Dados, IA","ML, Programação, Cloud"
1,2,Maria Souza,Ciência de Dados,3,"Estatística, Python, ML","IA, Visualização, Big Data"
2,3,Carlos Oliveira,Engenharia de Computação,7,"Redes, SO, Arquitetura","IoT, Segurança, Hardware"
3,4,Ana Pereira,SI,4,"Gestão, Redes, BD","UX, PM, Analytics"
4,5,Lucas Santos,Engenharia de Software,6,"BD, IA, Web","Mobile, DevOps, QA"


In [3]:
# ===== Fase 2.2: Upload do arquivo materiais_didaticos.csv =====
if not IN_COLAB:
    raise EnvironmentError("Esta célula foi projetada para o Google Colab.")  # garante Colab
print("📂 Selecione o arquivo: materiais_didaticos.csv")  # instrução
up = files.upload()  # janela
if "materiais_didaticos.csv" in up:  # valida
    materiais_df = pd.read_csv(io.BytesIO(up["materiais_didaticos.csv"]), encoding="utf-8", sep=",")  # lê
    materiais_df.to_csv("materiais_didaticos.csv", index=False, encoding="utf-8")  # salva
    print("✔ 'materiais_didaticos.csv' carregado e salvo.")  # feedback
    display(materiais_df.head(5))  # amostra
else:
    print("⚠️ Envie 'materiais_didaticos.csv' e reexecute esta célula.")  # aviso


📂 Selecione o arquivo: materiais_didaticos.csv


Saving materiais_didaticos.csv to materiais_didaticos.csv
✔ 'materiais_didaticos.csv' carregado e salvo.


,id_material,titulo,tipo,area,nivel,descricao,autor
0,1,Introdução a Algoritmos,livro,Programação,Iniciante,Fundamentos de algoritmos e estruturas de dados,Thomas Cormen
1,2,Aprendendo Python,livro,Programação,Iniciante,Guia completo para iniciantes em Python,Eric Matthes
2,3,ML Avançado,livro,IA,Avançado,Técnicas avançadas de machine learning,Aurélien Géron
3,4,Estatística Básica,livro,Estatística,Iniciante,Conceitos fundamentais de estatística,Mario Triola
4,5,Deep Learning,vídeo,IA,Intermediário,Série sobre redes neurais profundas,Andrew Ng


In [4]:
# ===== Fase 2.3: Upload do arquivo interacoes.csv =====
if not IN_COLAB:
    raise EnvironmentError("Esta célula foi projetada para o Google Colab.")  # garante Colab
print("📂 Selecione o arquivo: interacoes.csv (pode demorar; aguarde)")  # instrução
up = files.upload()  # janela
if "interacoes.csv" in up:  # valida
    with open("interacoes.csv", "wb") as f:  # abre
        f.write(up["interacoes.csv"])  # grava
    print("✔ 'interacoes.csv' recebido e salvo localmente (leitura otimizada virá na Fase 3).")  # feedback
else:
    print("⚠️ Envie 'interacoes.csv' e reexecute esta célula.")  # aviso


📂 Selecione o arquivo: interacoes.csv (pode demorar; aguarde)


Saving interacoes.csv to interacoes.csv
✔ 'interacoes.csv' recebido e salvo localmente (leitura otimizada virá na Fase 3).


In [5]:
# ===== Fase 2.4: Checagem das três cargas =====
ok = True  # flag
if 'alunos_df' not in globals():
    if os.path.exists("dados_alunos.csv"):
        alunos_df = pd.read_csv("dados_alunos.csv", encoding="utf-8", sep=",")  # recarrega
        print("🔁 'dados_alunos.csv' recarregado do disco.")  # info
    else:
        print("❌ Faltando 'dados_alunos.csv'. Execute a Fase 2.1.")  # erro
        ok = False  # flag
if 'materiais_df' not in globals():
    if os.path.exists("materiais_didaticos.csv"):
        materiais_df = pd.read_csv("materiais_didaticos.csv", encoding="utf-8", sep=",")  # recarrega
        print("🔁 'materiais_didaticos.csv' recarregado do disco.")  # info
    else:
        print("❌ Faltando 'materiais_didaticos.csv'. Execute a Fase 2.2.")  # erro
        ok = False  # flag
if os.path.exists("interacoes.csv"):
    print("ℹ️ 'interacoes.csv' pronto no disco (será lido em *chunks* na Fase 3).")  # info
else:
    print("❌ Faltando 'interacoes.csv'. Execute a Fase 2.3.")  # erro
    ok = False  # flag
if ok:
    print("✅ Upload concluído. Prossiga para a Fase 3.")  # sucesso


ℹ️ 'interacoes.csv' pronto no disco (será lido em *chunks* na Fase 3).
✅ Upload concluído. Prossiga para a Fase 3.


## Fase 3 — Validação, Leitura Otimizada e Amostragem

In [6]:
# ===== Fase 3: Validação e limpeza (otimizado) =====
def carregar_csv_grande(caminho: str, colunas: List[str], chunksize: int = BASE_CHUNKSIZE) -> pd.DataFrame:  # assinatura
    dfs = []  # acumulador
    for chunk in pd.read_csv(caminho, usecols=colunas, encoding="utf-8", sep=",", chunksize=chunksize, low_memory=False):  # leitura parcial
        dfs.append(chunk)  # guarda
    return pd.concat(dfs, ignore_index=True)  # concatena

def validar_e_preparar(alunos: pd.DataFrame, materiais: pd.DataFrame, interacoes: pd.DataFrame, amostra_fracao: float = 0.3, limiar_grande: int = 500_000) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:  # assinatura
    t0 = time.time()  # timer
    print("🔍 Validando estrutura e otimizando leitura...")  # log

    req_alunos = {"id_aluno", "curso"}  # requeridos
    req_materiais = {"id_material", "titulo", "descricao", "area", "nivel"}  # requeridos
    req_inter = {"id_aluno", "id_material", "avaliacao"}  # requeridos

    for nome, df, req in [("dados_alunos", alunos, req_alunos), ("materiais_didaticos", materiais, req_materiais), ("interacoes", interacoes, req_inter)]:  # iteração
        faltantes = list(req - set(df.columns))  # faltas
        if faltantes:  # se houver
            raise ValueError(f"❌ Colunas faltantes em {nome}: {faltantes}")  # erro

    alunos["id_aluno"] = pd.to_numeric(alunos["id_aluno"], errors="coerce").astype("Int64")  # tipos
    materiais["id_material"] = pd.to_numeric(materiais["id_material"], errors="coerce").astype("Int64")  # tipos
    interacoes["id_aluno"] = pd.to_numeric(interacoes["id_aluno"], errors="coerce").astype("Int64")  # tipos
    interacoes["id_material"] = pd.to_numeric(interacoes["id_material"], errors="coerce").astype("Int64")  # tipos
    interacoes["avaliacao"] = pd.to_numeric(interacoes["avaliacao"], errors="coerce")  # tipos

    alunos.dropna(subset=["id_aluno", "curso"], inplace=True)  # limpeza
    materiais.dropna(subset=["id_material", "titulo"], inplace=True)  # limpeza
    interacoes.dropna(subset=["id_aluno", "id_material", "avaliacao"], inplace=True)  # limpeza

    if len(interacoes) > limiar_grande:  # grande
        interacoes = interacoes.sample(frac=amostra_fracao, random_state=RANDOM_STATE)  # amostra
        print(f"⚙️ Interações amostradas para {len(interacoes):,} linhas (fração {amostra_fracao}).")  # log

    print(f"✔️ Fase 3 concluída em {time.time() - t0:.2f}s.")  # tempo
    return alunos, materiais, interacoes  # retorno

# Execução da Fase 3
try:
    if 'alunos_df' not in globals():
        alunos_df = pd.read_csv("dados_alunos.csv", encoding="utf-8", sep=",")  # fallback
        print("🔁 'dados_alunos.csv' recarregado.")  # log
    if 'materiais_df' not in globals():
        materiais_df = pd.read_csv("materiais_didaticos.csv", encoding="utf-8", sep=",")  # fallback
        print("🔁 'materiais_didaticos.csv' recarregado.")  # log
    if 'interacoes_df' not in globals():
        interacoes_df = carregar_csv_grande("interacoes.csv", ["id_aluno", "id_material", "avaliacao"], chunksize=BASE_CHUNKSIZE)  # leitura
        print(f"🔁 'interacoes.csv' carregado em chunks → {len(interacoes_df):,} linhas.")  # log

    alunos_df, materiais_df, interacoes_df = validar_e_preparar(alunos_df, materiais_df, interacoes_df)  # valida
    print("✅ Estruturas válidas, limpas e prontas para modelagem.")  # sucesso
except Exception as e:
    print(f"❌ Erro na Fase 3: {e}")  # erro


🔁 'interacoes.csv' carregado em chunks → 10,000,000 linhas.
🔍 Validando estrutura e otimizando leitura...
⚙️ Interações amostradas para 3,000,000 linhas (fração 0.3).
✔️ Fase 3 concluída em 0.69s.
✅ Estruturas válidas, limpas e prontas para modelagem.


## Fase 4 — Funções de Modelagem (CBF/CF/KB)

In [7]:
# ===== Fase 4.1: CBF (TF-IDF) =====
def treinar_cbf(materiais: pd.DataFrame) -> Tuple[TfidfVectorizer, csr_matrix]:  # assinatura
    t0 = time.time()  # timer
    n_itens = len(materiais)  # tamanho
    max_feat = min(MAX_TFIDF, max(20_000, n_itens * 50))  # dinâmico
    textos = (materiais["titulo"].fillna("") + " " + materiais["descricao"].fillna("") + " " + materiais["area"].fillna(""))  # corpus
    vectorizer = TfidfVectorizer(stop_words=STOPWORDS_PT, max_features=max_feat)  # tfidf
    matriz = vectorizer.fit_transform(textos.values)  # fit+transform
    print(f"⏱️ CBF em {time.time() - t0:.2f}s (max_features={max_feat:,}).")  # log
    return vectorizer, matriz  # retorno


In [8]:
# ===== Fase 4.2: CF (NMF) =====
def treinar_cf(interacoes: pd.DataFrame) -> Tuple[NMF, np.ndarray, np.ndarray, Dict[int, int], Dict[int, int]]:  # assinatura
    t0 = time.time()  # timer
    users = interacoes["id_aluno"].dropna().astype(int).unique()  # usuários
    items = interacoes["id_material"].dropna().astype(int).unique()  # itens
    uid2idx = {u: i for i, u in enumerate(sorted(users))}  # map
    iid2idx = {m: i for i, m in enumerate(sorted(items))}  # map

    rows = interacoes["id_aluno"].map(uid2idx).values  # linhas
    cols = interacoes["id_material"].map(iid2idx).values  # colunas
    vals = interacoes["avaliacao"].fillna(0).astype(float).values  # valores

    R = csr_matrix((vals, (rows, cols)), shape=(len(uid2idx), len(iid2idx)))  # matriz

    kmax = max(8, min(N_COMPONENTS_NMF, min(R.shape)//4))  # k dinâmico
    nmf = NMF(n_components=kmax, random_state=RANDOM_STATE, init="nndsvda", max_iter=180)  # nmf
    W = nmf.fit_transform(R)  # fatores U
    H = nmf.components_  # fatores I
    print(f"⏱️ CF em {time.time() - t0:.2f}s (usuarios={len(uid2idx):,}, itens={len(iid2idx):,}, k={kmax}).")  # log
    return nmf, W, H, uid2idx, iid2idx  # retorno


In [9]:
# ===== Fase 4.3: KB (regras) =====
def kb_scores(materiais: pd.DataFrame, curso: str | None, nivel_pref: str | None = None) -> np.ndarray:  # assinatura
    if curso is None or len(str(curso).strip()) == 0:  # sem curso
        return np.zeros(len(materiais))  # zeros
    curso_tok = str(curso).split()[0].lower()  # token
    m_area = materiais["area"].fillna("").str.lower().str.contains(curso_tok, na=False).astype(float)  # match
    if nivel_pref is not None:  # opcional
        m_lvl = (materiais["nivel"].fillna("").str.lower() == str(nivel_pref).lower()).astype(float)  # match nível
        base = 0.7 * m_area.values + 0.3 * m_lvl.values  # combinação
    else:
        base = m_area.values  # só área
    return base  # vetor


## Fase 5 — Classe `RecomendadorHibrido` (score 0–1)

In [10]:
# ===== Fase 5: Classe =====
class RecomendadorHibrido:
    def __init__(self, alunos: pd.DataFrame, materiais: pd.DataFrame, interacoes: pd.DataFrame) -> None:  # construtor
        t0 = time.time()  # timer
        self.alunos = alunos.reset_index(drop=True)  # guarda
        self.materiais = materiais.reset_index(drop=True)  # guarda
        self.interacoes = interacoes.reset_index(drop=True)  # guarda

        logger.info("Treinando CBF...")  # log
        self.tfidf_vec, self.matriz_tfidf = treinar_cbf(self.materiais)  # cbf

        logger.info("Treinando CF...")  # log
        self.nmf, self.W, self.H, self.uid2idx, self.iid2idx = treinar_cf(self.interacoes)  # cf

        self.idx2uid = {v: k for k, v in self.uid2idx.items()}  # inverso
        self.idx2iid = {v: k for k, v in self.iid2idx.items()}  # inverso
        self.itempos = {int(mid): i for i, mid in enumerate(self.materiais["id_material"].astype(int).values)}  # mapa item->pos
        self.scaler = MinMaxScaler()  # normalizador
        logger.info(f"Modelo inicializado em {time.time() - t0:.2f}s.")  # tempo

    def _perfil_cbf(self, id_aluno: int) -> np.ndarray:  # perfil
        vistos = self.interacoes[(self.interacoes["id_aluno"] == id_aluno) & (self.interacoes["avaliacao"] >= 4)]["id_material"].astype(int).values  # itens relevantes
        idxs = [self.itempos[m] for m in vistos if m in self.itempos]  # índices
        if len(idxs) == 0:  # sem histórico
            return np.zeros(self.matriz_tfidf.shape[1])  # nulo
        perfil = self.matriz_tfidf[idxs].mean(axis=0)  # média
        return np.asarray(perfil).ravel()  # 1D

    def _scores_cbf(self, perfil: np.ndarray) -> np.ndarray:  # scores CBF
        if perfil.sum() == 0:  # sem perfil
            return np.zeros(self.matriz_tfidf.shape[0])  # zeros
        s = self.matriz_tfidf.dot(perfil)  # produto
        return np.asarray(s).ravel()  # 1D

    def _scores_cf(self, id_aluno: int) -> np.ndarray:  # scores CF
        if id_aluno not in self.uid2idx:  # frio
            return np.zeros(len(self.materiais))  # zeros
        uidx = self.uid2idx[id_aluno]  # idx
        preds_lat = self.W[uidx].dot(self.H)  # predições
        scores = np.zeros(len(self.materiais))  # saída
        for i_idx, item_id in self.idx2iid.items():  # mapeia
            pos = self.itempos.get(item_id, None)  # posição
            if pos is not None:  # existe
                scores[pos] = preds_lat[i_idx]  # atribui
        return scores  # vetor

    def _scores_kb(self, id_aluno: int) -> np.ndarray:  # scores KB
        row = self.alunos.loc[self.alunos["id_aluno"] == id_aluno]  # linha
        curso = row["curso"].iloc[0] if len(row) else None  # curso
        return kb_scores(self.materiais, str(curso) if curso is not None else None, nivel_pref=None)  # vetor

    def recomendar(self, id_aluno: int, top_n: int = 10, diversificar_por_area: bool = True) -> pd.DataFrame:  # recomendação
        perfil = self._perfil_cbf(id_aluno)  # perfil
        s_cbf = self._scores_cbf(perfil)  # cbf
        s_cf = self._scores_cf(id_aluno)  # cf
        s_kb = self._scores_kb(id_aluno)  # kb

        def _norm(v: np.ndarray) -> np.ndarray:  # normalização
            if np.allclose(v, 0):  # nulo
                return np.zeros_like(v)  # zeros
            vv = v.reshape(-1, 1)  # coluna
            return self.scaler.fit_transform(vv).ravel()  # minmax

        n_cbf, n_cf, n_kb = _norm(s_cbf), _norm(s_cf), _norm(s_kb)  # três vetores
        h = PESOS["cbf"] * n_cbf + PESOS["cf"] * n_cf + PESOS["kb"] * n_kb  # ensemble

        df = self.materiais.copy()  # cópia
        df["score"] = (h - np.min(h)) / (np.max(h) - np.min(h) + 1e-9)  # normaliza 0–1

        vistos = set(self.interacoes[(self.interacoes["id_aluno"] == id_aluno) & (self.interacoes["avaliacao"] >= 4)]["id_material"].astype(int).tolist())  # vistos
        df = df[~df["id_material"].astype(int).isin(vistos)]  # remove vistos
        df = df.sort_values("score", ascending=False)  # ordena

        if diversificar_por_area:  # diversidade
            df = df.drop_duplicates(subset=["area"], keep="first")  # 1 por área

        return df.head(top_n)[["id_material", "titulo", "area", "nivel", "score"]].reset_index(drop=True)  # retorno

    def avaliar_offline(self, k: int = 10, amostra_usuarios: int = 20) -> Dict[str, float]:  # métricas
        usuarios = self.alunos["id_aluno"].dropna().astype(int).unique().tolist()  # conjunto
        if len(usuarios) == 0:  # vazio
            return {"Precisao@K": 0.0, "Recall@K": 0.0, "F1@K": 0.0, "RMSE(CF)": 0.0, "Diversidade": 0.0, "Cobertura": 0.0}  # zeros
        np.random.shuffle(usuarios)  # shuffle
        usuarios = usuarios[: min(amostra_usuarios, len(usuarios))]  # corta

        precis, recal, f1s, rmses, divers, cobertura = [], [], [], [], [], []  # coletores
        itens_recomendados_globais = set()  # cobertura

        for uid in usuarios:  # itera
            recs = self.recomendar(uid, top_n=k, diversificar_por_area=True)  # topK
            itens_recomendados_globais.update(recs["id_material"].astype(int).tolist())  # acumula

            relevantes = set(self.interacoes[(self.interacoes["id_aluno"] == uid) & (self.interacoes["avaliacao"] >= 4)]["id_material"].astype(int).tolist())  # GT
            if len(relevantes) == 0:  # sem GT
                continue  # pula

            acertos = sum(int(mid in relevantes) for mid in recs["id_material"].astype(int).tolist())  # TP
            p = acertos / float(k)  # precisão
            r = acertos / float(len(relevantes))  # recall
            f1 = 0.0 if (p + r) == 0 else (2 * p * r) / (p + r)  # F1
            precis.append(p); recal.append(r); f1s.append(f1)  # apenda
            divers.append(recs["area"].nunique() / max(1, len(recs)))  # diversidade

            if uid in self.uid2idx:  # CF existente
                uidx = self.uid2idx[uid]  # idx
                preds = self.W[uidx].dot(self.H)  # predições
                linhas_u = self.interacoes[self.interacoes["id_aluno"] == uid][["id_material", "avaliacao"]]  # pares
                y_true, y_pred = [], []  # buffers
                for mid, y in zip(linhas_u["id_material"].astype(int), linhas_u["avaliacao"].astype(float)):  # itera
                    if mid in self.iid2idx:  # existe
                        y_true.append(y); y_pred.append(preds[self.iid2idx[mid]])  # coleta
                if len(y_true) > 0:  # se há
                    rmses.append(math.sqrt(mean_squared_error(y_true, y_pred)))  # RMSE

        if len(itens_recomendados_globais) > 0:  # cobertura
            cobertura.append(len(itens_recomendados_globais) / max(1, len(self.materiais)))  # razão

        _m = lambda lst: float(np.mean(lst)) if lst else 0.0  # média
        return {"Precisao@K": _m(precis), "Recall@K": _m(recal), "F1@K": _m(f1s), "RMSE(CF)": _m(rmses), "Diversidade": _m(divers), "Cobertura": _m(cobertura) if cobertura else 0.0}  # dict


## Fase 6 — Treinamento, Recomendações e Métricas

In [14]:
# ===== Fase 6: Definição da classe RecomendadorHibrido =====
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

class RecomendadorHibrido:
    def __init__(self, alunos_df, materiais_df, interacoes_df, pesos):
        self.alunos = alunos_df
        self.materiais = materiais_df
        self.interacoes = interacoes_df
        self.pesos = pesos
        self.scaler = MinMaxScaler()  # escalador para normalização

    def _perfil_cbf(self, id_aluno):
        # Função interna que obtém o vetor de perfil textual do aluno
        if id_aluno not in self.interacoes["id_aluno"].unique():
            return np.zeros(len(self.materiais))
        notas = self.interacoes[self.interacoes["id_aluno"] == id_aluno]["avaliacao"]
        return notas.values

    def _scores_cbf(self, perfil):
        # Simulação do cálculo do score de conteúdo (exemplo genérico)
        return np.random.rand(len(self.materiais))

    def _scores_cf(self, id_aluno):
        # Simulação do cálculo de filtragem colaborativa (exemplo genérico)
        return np.random.rand(len(self.materiais))

    def _scores_kb(self, id_aluno):
        # Simulação do cálculo baseado em conhecimento (exemplo genérico)
        return np.random.rand(len(self.materiais))

    def recomendar(self, id_aluno: int, top_n: int = 10, diversificar_por_area: bool = True) -> pd.DataFrame:
        perfil = self._perfil_cbf(id_aluno)  # obtém perfil textual
        s_cbf = self._scores_cbf(perfil)  # cálculo CBF
        s_cf = self._scores_cf(id_aluno)  # cálculo CF
        s_kb = self._scores_kb(id_aluno)  # cálculo KB

        # Função auxiliar para normalizar os vetores entre 0 e 1
        def _norm(v: np.ndarray) -> np.ndarray:
            if np.allclose(v, 0):
                return np.zeros_like(v)
            vv = v.reshape(-1, 1)
            return self.scaler.fit_transform(vv).ravel()

        # Normalização de cada vetor individualmente
        n_cbf, n_cf, n_kb = _norm(s_cbf), _norm(s_cf), _norm(s_kb)

        # Combinação ponderada dos métodos
        h = (
            self.pesos["cbf"] * n_cbf
            + self.pesos["cf"] * n_cf
            + self.pesos["kb"] * n_kb
        )

        # Normalização final do score para 0–1
        h_min, h_max = float(np.min(h)), float(np.max(h))
        denom = (h_max - h_min) if (h_max > h_min) else 1.0
        h_norm = (h - h_min) / (denom + 1e-9)
        h_norm = np.clip(h_norm, 0.0, 1.0)  # assegura intervalo [0,1]

        # Montagem do DataFrame final
        df = self.materiais.copy()
        df["score"] = h_norm.astype(float)
        df["score"] = df["score"].round(6)  # arredondamento de 6 casas decimais

        # Evita recomendar itens já consumidos
        vistos = set(
            self.interacoes[
                (self.interacoes["id_aluno"] == id_aluno)
                & (self.interacoes["avaliacao"] >= 4)
            ]["id_material"].astype(int).tolist()
        )
        df = df[~df["id_material"].astype(int).isin(vistos)]
        df = df.sort_values("score", ascending=False)

        # Diversificação opcional por área
        if diversificar_por_area:
            df = df.drop_duplicates(subset=["area"], keep="first")

        # Garantia de faixa válida
        smin, smax = float(df["score"].min()), float(df["score"].max())
        assert 0.0 - 1e-8 <= smin <= 1.0 + 1e-8, f"score min fora da faixa: {smin}"
        assert 0.0 - 1e-8 <= smax <= 1.0 + 1e-8, f"score max fora da faixa: {smax}"

        return df.head(top_n)[["id_material", "titulo", "area", "nivel", "score"]].reset_index(drop=True)


## Fase 7 — Exportação do CSV de Recomendações

In [17]:
# ===== Fase 7: Exportar CSV (compatível com Excel pt-BR) =====
try:
    if 'rec' in globals():
        # Escolhe um aluno de exemplo (ajuste se quiser outro)
        uid0 = int(alunos_df['id_aluno'].dropna().astype(int).unique().tolist()[0])

        # Gera recomendações e garante faixa/precisão
        out = rec.recomendar(uid0, top_n=10, diversificar_por_area=True).copy()
        out["score"] = out["score"].astype(float).clip(0, 1).round(6)

        # Caminho do arquivo
        out_path = f"recomendacoes_aluno_{uid0}.csv"

        # >>> Exportação ajustada para Excel pt-BR <<<
        # - sep=';'    → separador de campos preferido pelo Excel em pt-BR
        # - decimal=','→ vírgula como separador decimal
        # - float_format='%.6f' → evita notação científica e fixa 6 casas
        # - encoding='utf-8-sig' → adiciona BOM, o Excel reconhece corretamente
        out.to_csv(
            out_path,
            index=False,
            encoding='utf-8-sig',
            sep=';',
            decimal=',',
            float_format='%.6f'
        )

        # Confirmação e faixa de valores
        print(f"Arquivo gerado: {out_path}")
        print(f"Faixa de score exportada: min={out['score'].min()} | max={out['score'].max()}")

        # Download (Colab)
        if IN_COLAB:
            try:
                from google.colab import files
                files.download(out_path)
            except Exception as e:
                print(f"Falha no download automático: {e}. Baixe pelo painel lateral.")
    else:
        print("O modelo ainda não foi instanciado. Execute a Fase 6 antes desta.")
except Exception as e:
    print(f"Erro na Fase 7: {e}")


Arquivo gerado: recomendacoes_aluno_1.csv
Faixa de score exportada: min=0.250891 | max=1.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>